In [1]:
import numpy as np
import pandas as pd
from equiflow import *

In [2]:
np.random.seed(42)
n = 100000
data = {
    'age': np.random.randint(10, 80, size=n),
    'sofa': np.random.choice([0,1,2,3,4,5,6,7,8,9,10,15, np.nan], size=n),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Hispanic', None],
                             size=n),
    'sex': np.random.choice(['Male', 'Female'], size=n),
    'english': np.random.choice(['Fluent', 'Limited', np.nan, None], size=n),
}

for i in range(1, 11):
    data[f'var{i}'] = np.random.randn(n)

df = pd.DataFrame(data)

In [7]:
eqfl = EquiFlow(data=df,
                initial_cohort_label='in MIMIC-IV',
                categorical=['sex','race'],
                normal=['age'],
                nonnormal=['sofa'],
                rename={'sofa': 'SOFA Score'},
                )

eqfl.add_exclusion(
    df.english.notnull(),
    exclusion_reason='missing English Proficiency',
    new_cohort_label='with English Proficiency data'
)

eqfl.view_table_characteristics()

Cohort                
                                                0               1
Variable                 Value                                   
Overall                                   100,000          49,914
sex, N (%)               Female     49,938 (49.9)   24,975 (50.0)
                         Male       50,062 (50.1)   24,939 (50.0)
                         Missing          0 (0.0)         0 (0.0)
race, N (%)              Asian      20,035 (20.0)   10,119 (20.3)
                         White      20,028 (20.0)    9,917 (19.9)
                         Hispanic   19,900 (19.9)    9,985 (20.0)
                         Black      20,008 (20.0)    9,937 (19.9)
                         Missing    20,029 (20.0)    9,956 (19.9)
age, Mean ± SD                        44.4 ± 20.2     44.3 ± 20.2
                         Missing          0 (0.0)         0 (0.0)
SOFA Score, Median [IQR]           5.0 [2.0, 9.0]  5.0 [2.0, 9.0]
                         Missing      7,605 (7.6)     3,770 (7.6)

In [4]:
eqfl = EquiFlow(data=df,
                initial_cohort_label='in MIMIC-IV',
                categorical=['sex','race'],
                normal=['age'],
                nonnormal=['sofa'],
                rename={'sofa': 'SOFA Score'},
                )

eqfl.add_exclusion(
    df.english.notnull(),
    exclusion_reason='missing English Proficiency',
    new_cohort_label='with English Proficiency data'
)

eqfl.plot_flows()

In [8]:
eqfl = EquiFlow(df,
                initial_cohort_label='MIMIC-IV',
                categorical=['english'],
                normal=['age'],
                nonnormal=['sofa'],
                # rename={
                #     'sofa': 'SOFA',
                #     'age': 'Age',
                #     'english': 'English Proficiency',
                # },
                )

eqfl.add_exclusion(
    mask=df.english.notnull(),
    exclusion_reason='missing English Proficiency',
    new_cohort_label='with English Proficiency data'
)

# add a filter for age
eqfl.add_exclusion(
    mask=df.age >= 18,
    exclusion_reason='age < 18',
    new_cohort_label='adults only'
)                    

eqfl.plot_flows(
    legend=True
)

In [9]:
eqfl.view_table_flows()

Cohort Flow,0 to 1,1 to 2
,,
"Initial, n","100,000","49,914"
"Removed, n","50,086","5,777"
"Result, n","49,914","44,137"


In [10]:
eqfl.view_table_characteristics()

Cohort                                
                                         0               1               2
Variable           Value                                                  
Overall                            100,000          49,914          44,137
english, N (%)     Limited   24,924 (24.9)   24,924 (49.9)   21,987 (49.8)
                   Fluent    24,990 (25.0)   24,990 (50.1)   22,150 (50.2)
                   Missing   50,086 (50.1)         0 (0.0)         0 (0.0)
age, Mean ± SD                 44.4 ± 20.2     44.3 ± 20.2     48.4 ± 17.9
                   Missing         0 (0.0)         0 (0.0)         0 (0.0)
sofa, Median [IQR]          5.0 [2.0, 9.0]  5.0 [2.0, 9.0]  5.0 [2.0, 9.0]
                   Missing     7,605 (7.6)     3,770 (7.6)     3,311 (7.5)

In [12]:
eqfl.view_table_drifts(drifts_by_class=True)

Cohort Flow,0 to 1,1 to 2
english,1.4,0
age,-0.0,0.2
sofa,0.0,0.0


In [13]:
eqfl.view_table_drifts(drifts_by_class=False)

Cohort Flow      0 to 1 1 to 2
Variable Value                
Overall                       
english  Limited    0.5    0.0
         Fluent     0.5    0.0
age                -0.0    0.2
sofa                0.0    0.0

In [14]:
data_0 = df.copy()
data_1 = data_0.loc[data_0.english.notnull()]
data_2 = data_1.loc[data_1.sofa.notnull()]

In [15]:
TableFlows(
    dfs = [data_0, data_1, data_2],
    label_suffix=True,
    thousands_sep=False,
).view()

Cohort Flow,0 to 1,1 to 2
,,
"Initial, n",100000,49914
"Removed, n",50086,3770
"Result, n",49914,46144


In [16]:
TableCharacteristics(
    dfs = [data_0, data_1, data_2],
    categorical = ['race','sex', 'english'],
    nonnormal = ['sofa'],
    normal = ['age'],
    format_cat = 'N (%)',
    format_normal = 'Mean',
    format_nonnormal='Median [IQR]',
    missingness = True,
    decimals = 1,
    label_suffix = True,
    thousands_sep = True,
    rename={'race': 'Race and Ethnicity',
            'english': 'English Proficiency',
            'sex':'Sex',
            'sofa': 'SOFA',
            'age': 'Age',  
            }
).view()

Cohort                  \
                                                  0               1   
Variable                   Value                                      
Overall                                     100,000          49,914   
Race and Ethnicity, N (%)  Asian      20,035 (20.0)   10,119 (20.3)   
                           White      20,028 (20.0)    9,917 (19.9)   
                           Hispanic   19,900 (19.9)    9,985 (20.0)   
                           Black      20,008 (20.0)    9,937 (19.9)   
                           Missing    20,029 (20.0)    9,956 (19.9)   
Sex, N (%)                 Female     49,938 (49.9)   24,975 (50.0)   
                           Male       50,062 (50.1)   24,939 (50.0)   
                           Missing          0 (0.0)         0 (0.0)   
English Proficiency, N (%) Limited    24,924 (24.9)   24,924 (49.9)   
                           Fluent     24,990 (25.0)   24,990 (50.1)   
                           Missing    50,086 (50.1)         0 (0.0)   
Age, Mean                                      44.4            44.3   
                           Missing          0 (0.0)         0 (0.0)   
SOFA, Median [IQR]                   5.0 [2.0, 9.0]  5.0 [2.0, 9.0]   
                           Missing      7,605 (7.6)     3,770 (7.6)   

                                                     
                                                  2  
Variable                   Value                     
Overall                                      46,144  
Race and Ethnicity, N (%)  Asian       9,308 (20.2)  
                           White       9,191 (19.9)  
                           Hispanic    9,222 (20.0)  
                           Black       9,196 (19.9)  
                           Missing     9,227 (20.0)  
Sex, N (%)                 Female     23,069 (50.0)  
                           Male       23,075 (50.0)  
                           Missing          0 (0.0)  
English Proficiency, N (%) Limited    23,047 (49.9)  
                           Fluent     23,097 (50.1)  
                           Missing          0 (0.0)  
Age, Mean                                      44.3  
                           Missing          0 (0.0)  
SOFA, Median [IQR]                   5.0 [2.0, 9.0]  
                           Missing          0 (0.0)

In [19]:
TableDrifts(
    dfs=[data_0, data_1, data_2],
    categorical = ['race','sex', 'english'],
    nonnormal = ['sofa'],
    normal = ['age'],
    missingness = True,
    decimals = 3,
    rename={'race': 'Race and Ethnicity',
            'english': 'English Proficiency',
            'sex':'Sex',
            'sofa': 'SOFA',
            'age': 'Age',  
            }
).view()

Cohort Flow                  0 to 1 1 to 2
Variable            Value                 
Overall                                   
Race and Ethnicity  Asian     0.006  0.003
                    White     0.004  0.001
                    Hispanic  0.003    0.0
                    Black     0.003  0.001
Sex                 Female    0.002  0.001
                    Male      0.002  0.001
English Proficiency Limited   0.535    0.0
                    Fluent    0.536    0.0
Age                            -0.0   -0.0
SOFA                         -0.001    0.0